# Beyond Hello World, A Computer Vision Example


前回の演習では、自分が解決しようとしている問題を解決するためのニューラルネットワークを作成する方法を見ました。これは学習された動作の明示的な例を示していました。もちろん、この例では、機械学習を使ってXとYの関係を学習し、固定された値の集合に対してそれをすべての値に対して拡張するのではなく、関数Y=3x+1を直接書く方が簡単だったでしょうから、少しやりすぎでした。

しかし、このようなルールを書くのがもっと難しいシナリオ、例えばコンピュータビジョンの問題ではどうでしょうか？10種類の異なるタイプの服を含むデータセットから学習して、異なる服のアイテムを認識するシナリオを見てみましょう。

## Start Coding


まずはTensorFlowのインポートから始めましょう

In [ ]:
import tensorflow as tf
print(tf.__version__)



Fashion MNISTと呼ばれる共通のデータセットから服のアイテムを認識するためのニューラルネットワークを訓練します。このデータセットについての詳細は[こちら](https://github.com/zalandoresearch/fashion-mnist)を参照してください。

このデータセットには、10種類のカテゴリに分類された7万点の衣類が含まれています。衣類の各アイテムは28x28のグレースケールの画像になっています。ここでいくつかの例を見ることができます。

![alt text](https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/fashion-mnist-sprite.png)



Fashion MNISTのデータは、tf.keras datasets APIで直接利用できます。読み込みは以下のように行います。

In [ ]:
mnist = tf.keras.datasets.fashion_mnist



このオブジェクトで load_data を呼び出すと、2つのリストの2つのセットが得られます。これらは、衣類のアイテムとそのラベルを含むグラフィックスのトレーニング値とテスト値になります。

In [ ]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()



これらの値はどのように見えるでしょうか？トレーニング画像とトレーニングラベルを表示して見てみましょう...配列の異なるインデックスを表示したらどうなるでしょうか。例えばインデックス番号42ですが、これはインデックス番号が0のものとは異なるブーツの画像です。

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(training_images[0])
print(training_labels[0])
print(training_images[0])



また、数値の中のすべての値が0から255の間であることに気づくでしょう。ニューラルネットワークを学習する場合、様々な理由から、すべての値を0から1の間の値として扱う方が簡単です。そのことを、「**normalizing（正規化）**」と呼びます。幸運なことに、Pythonではループを作ることなく簡単に正規化することができます。方法は以下の通りです。

In [ ]:
training_images  = training_images / 255.0
test_images = test_images / 255.0



さて、なぜ2つのセット（トレーニングとテスト）があるのか不思議に思うかもしれません。冒頭の部分でこれについて話したのを覚えていますか？このアイデアは、トレーニング用に1つのデータセット。もうひとつは、モデルがまだ見ていない別のものになります。これはデータを分類するのにどれだけ優れているかを確認するためです。一般性を確認するために、トレーニンで使用していないデータで試す必要があることは理解いただけると思います。





では、モデルを設計してみましょう。ここには新しい概念がかなりありますが、心配しないでください。

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])



**Sequential**: ニューラルネットワークのレイヤーのシーケンスを定義します。

**Flatten**: 以前、プリントアウトした画像が四角かったのを覚えていますか？平坦化は、その正方形を1次元のセットに変換します。

**Dense**: ニューロンのレイヤーを追加します。

ニューロンの各層には、**activation function（活性化関数）** が必要です。オプションはたくさんありますが、今のところは以下のものを使ってください。

**Relu**は、「X>0ならXを返し、0なら0を返す」ということを意味します。結果として、0もしくはそれより大きな値を次のレイヤーに渡すだけです。

**Softmax**は値のセットを受け取り、効果的に一番大きな値を選びます。例えば、最後のレイヤの出力が [0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05] のようになっている場合、一番大きな値を探す手間を省き、[0,0,0,0,0,1,0,0,0,0] に変換してくれます。




次にやるべきことは、モデルが定義されたので、実際にそれを構築することです。先ほどと同じようにオプティマイザと損失関数を使ってコンパイルします -- そして、**model.fit **を呼び出してトレーニングを行います -- トレーニングデータから得られる推測値をラベルに一致するように学習させます -- つまり、トレーニングデータと実際のラベルの間の関係を把握させます。将来、学習データのようなデータがあれば、そのデータがどのように見えるかを予測することができます。

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)



トレーニングが完了したら、最後のエポックの後に精度の値が表示されるはずです。0.9098のように見えるかもしれません。これは、ニューラルネットワークがトレーニングデータを分類するのに約91%の精度であることを示しています。つまり、画像とラベルの間のパターンマッチを計算して、91%の確率で機能していることになります。素晴らしいことではありませんが、5エポックの間だけ訓練されていて、かなり素早く行われたことを考えると、悪くはありません。

それでは、まだ見ていないデータではどうでしょうか？そのためにテスト画像があります。model.evaluateを呼び出して、2つのセットを渡すことができます。試してみましょう。

In [ ]:
model.evaluate(test_images, test_labels)



私の場合、約0.8838の精度が返されましたが、これは約88%の精度だったことを意味します。予想されていたように、これはおそらく、*見ていない*データでは、訓練されたデータと同じようにうまくいかないでしょう。このコースでは、これを改善する方法を見ていきます。

さらに探求するには、以下の演習を試してみてください。

# Exploration Exercises

###Exercise 1:
この最初の演習では、以下のコードを実行してください。テスト画像のそれぞれについて分類を行います。そして分類結果の最初のエントリを表示します。出力については、数字のリストとなります。これはなぜだと思いますか、また、これらの数字は何を表していると思いますか？

In [ ]:
classifications = model.predict(test_images)

print(classifications[0])

ヒント: print(test_labels[0])を実行してみてください。 このリストがなぜこのように見えるのか理解できましたか？

In [ ]:
print(test_labels[0])

### このリストは何を表しているのでしょうか？
1.   無意味な10個のランダムな値
2.   コンピュータが作った最初の10分類
3.   10クラスそれぞれの確率


####回答
正解は(3)です。

### このリストを見て、その商品がアンクルブーツであることがわかるのはなぜですか?


1.   その質問に答えるのに十分な情報がありません。
2.   リストの10番目の要素が最も大きく、アンクルブーツには9のラベルが付けられています。
3.   アンクルブーツはラベル9で、リストには0->9の要素があります。



####回答
正解は(2)です。

##Exercise 2: 
それでは、あなたのモデルの層を見てみましょう。512個のニューロンを持つ全結合層の値を変えて実験してみてください。損失、トレーニング時間などについて変化が見られたでしょうか？それはなぜだと思いますか？



In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

###質問1. ニューロンを1024個に増やした場合の影響は？

1. トレーニングに時間がかかるが、より正確なトレーニングができる
2. トレーニングに時間がかかるが、精度には影響なし
3. トレーニング時間に変化がないが、より正確です。


####回答
正解は(1)です。

##Exercise 3: 

Flatten()レイヤーを削除するとどうなるか。なぜそうなると思いますか？

データの形状に関するエラーが発生します。今は漠然としているように見えるかもしれませんが、ネットワークの最初のレイヤーはデータと同じ形状であるべきだという経験則を補強しています。今、私たちのデータは28x28の画像で、28のニューロンからなる28層のレイヤーは不可能なので、28,28を784x1に「平坦化」する方が理にかなっています。 自分たちで処理するためにすべてのコードを書かなくても、最初にFlatten()レイヤーを追加して、後で配列がモデルにロードされることで、自動的に平坦化されて処理を続けることができます。

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0


model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# This version has the 'flatten' removed. Replace the above with this one to see the error.
#model = tf.keras.models.Sequential([tf.keras.layers.Dense(64, activation=tf.nn.relu),
#                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])


model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

##練習4. 

最終的な（出力される）レイヤーを考えてみましょう。なぜ10個もあるのでしょうか？10個とは異なる量を持っていたらどうなるでしょうか？例えば、5個にするとどうなるでしょうか。

想定外の値を見つけるとすぐにエラーになります。もう一つの経験則 -- 最後の層のニューロンの数は、あなたが分類しているクラスの数と一致しなければなりません。この場合、0-9の数字なので、最終層には10個のニューロンがあるはずです。

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# Replace the above model definiton with this one to see the network with 5 output layers
# And you'll see errors as a result!
# model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
#                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
#                                    tf.keras.layers.Dense(5, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

##練習5. 

ネットワークにレイヤーを追加した場合の効果を考えてみましょう。512個のニューロンを持つ全結合層と10の最終層の間にもう1つのレイヤーを追加するとどうなるでしょうか。

回答：大きな影響はありません -- これは比較的単純なデータだからです。はるかに複雑なデータ（次のレッスンで確認するカラー画像の花の分類等）では、多くの場合、追加のレイヤーが必要になります。

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

#Exercise 6: 

エポック数を増減してトレーニングをした場合の影響を考えてみましょう。

15エポックを試してみてください -- おそらく、5エポックのモデルよりもはるかに良い損失を持つモデルが得られるでしょう
30エポックを試してみてください -- 損失値の減少が止まったり、時には増加したりするのがわかるかもしれません。これは「過学習」と呼ばれる副作用です。これはニューラルネットワークをトレーニングするときには意識しておかなくてはならないものです。いずれにしても、損失が改善されないのであれば、時間をかけてトレーニングしても意味がないですよね。

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=30)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[34])
print(test_labels[34])

#Exercise 7: 

トレーニングの前に、データを正規化して、0-255の値から0-1の値にしました。これを削除した場合、どのような影響があるでしょうか？
以下のコードは、それを試すためのものです。なぜ異なる結果が得られると思いますか？

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
# To experiment with removing normalization, comment out the following 2 lines
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

#Exercise 8: 

以前、エポックを追加してトレーニングをしていたときに、損失関数の出力が変わるかもしれないということがありました。トレーニングの完了を待つ間、「希望の精度に達したときにトレーニングを止められたらいいのに」と思ったかもしれません。たとえば、95%の精度がでれば十分な時にそれが、たった3エポックで実現できたとすると、残りのエポックが完了するまでどうして待たなくてはならないのかと思うかもしれません。どうすれば解決できるでしょうか。それを解決するためにコールバックがあります。実際に見てみましょう。

In [ ]:
import tensorflow as tf
print(tf.__version__)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if((logs['accuracy'] > 0.9)):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])


